# Fake news classifier using LSTM
Dataset was taken from Kaggle: https://www.kaggle.com/c/fake-news/overview

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('datasets/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

### Data Cleaning   

In [6]:
#Drop NA values
df = df.dropna()
df.shape

(18285, 5)

In [7]:
# Get the independent features

X = df.drop('label',axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [8]:
#Get the dependent features

y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

### Installing Tensorflow dependancies

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.7.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
#Vocabulary size 
voc_size = 5000


In [15]:
messages = X.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
### Data Preprocessing  
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
   
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)



In [19]:
len(corpus)

18285

In [20]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

### Onehot Representation

In [21]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[3924, 438, 4621, 1162, 4022, 415, 2902, 4032, 4889, 3726],
 [4000, 1017, 4884, 500, 3699, 691, 384],
 [3084, 3297, 3799, 4470],
 [365, 4438, 2843, 1419, 4802, 483],
 [253, 3699, 1906, 193, 1939, 2014, 3699, 3698, 973, 2170],
 [479,
  4218,
  399,
  1774,
  3872,
  3265,
  3001,
  4261,
  2008,
  1791,
  2226,
  123,
  3705,
  2133,
  384],
 [1538, 712, 2789, 4905, 2211, 4183, 3817, 3435, 2924, 2562, 3119],
 [1464, 39, 4738, 3555, 294, 278, 3265, 403, 2924, 2562, 3119],
 [1970, 4302, 1435, 4821, 3548, 3186, 4465, 310, 3265, 3391],
 [1723, 1839, 3772, 4014, 2504, 4149, 4813, 1698],
 [2771, 1782, 4617, 4213, 399, 1153, 687, 3846, 4230, 3671, 2918],
 [1419, 3217, 4022, 3186, 3265, 294],
 [557, 297, 4227, 4994, 4840, 1555, 4394, 3007, 2050],
 [836, 1455, 4825, 2021, 3070, 4048, 1893, 2924, 2562, 3119],
 [2777, 634, 2009, 1215, 1221, 2924, 2562, 3119],
 [1681, 1244, 750, 4014, 958, 948, 2961, 3474, 4088, 1534],
 [1913, 466, 1017],
 [2447, 4800, 4733, 2778, 3265, 3428, 1603, 384],
 [4409, 2

### Embedding Representation

In [22]:
#Fill in the space with zeros so that all sentences are of the same lengths
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding = 'pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4032, 4889, 3726],
       [   0,    0,    0, ..., 3699,  691,  384],
       [   0,    0,    0, ..., 3297, 3799, 4470],
       ...,
       [   0,    0,    0, ..., 2924, 2562, 3119],
       [   0,    0,    0, ..., 4811,  637, 3749],
       [   0,    0,    0, ...,  780, 2065, 1023]])

In [25]:
dim = 40

In [35]:
model = Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)
X_train.shape

(14628, 20)

### Model Training

In [60]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=1)

Epoch 1/10
14628/14628 [==============================] - 102s 7ms/step - loss: 0.0878 - accuracy: 0.9675 - val_loss: 0.2671 - val_accuracy: 0.9150
Epoch 2/10
14628/14628 [==============================] - 98s 7ms/step - loss: 0.0637 - accuracy: 0.9765 - val_loss: 0.2348 - val_accuracy: 0.9125
Epoch 3/10
14628/14628 [==============================] - 97s 7ms/step - loss: 0.0501 - accuracy: 0.9825 - val_loss: 0.2846 - val_accuracy: 0.9128
Epoch 4/10
14628/14628 [==============================] - 101s 7ms/step - loss: 0.0376 - accuracy: 0.9867 - val_loss: 0.3167 - val_accuracy: 0.8996
Epoch 5/10
14628/14628 [==============================] - 95s 6ms/step - loss: 0.0306 - accuracy: 0.9899 - val_loss: 0.3343 - val_accuracy: 0.9078
Epoch 6/10
14628/14628 [==============================] - 92s 6ms/step - loss: 0.0264 - accuracy: 0.9915 - val_loss: 0.3906 - val_accuracy: 0.9128
Epoch 7/10
14628/14628 [==============================] - 93s 6ms/step - loss: 0.0183 - accuracy: 0.9936 - val_loss:

In [61]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics and Accuracy

In [62]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [63]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2082,    0],
       [1575,    0]], dtype=int64)

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5693191140278917